# Pre-processing pipeline for spikeglx sessions, zebra finch
- For every run in the session:
 - Load the recordings
 - Extract wav chan with micrhopohone and make a wav chan with the nidq syn signal
 - Get the sync events for the nidq sync channel
 
 - Do bout detection
 
In another notebook, bout detection is curated
- Left to decide where to:
    - Sort spikes
    - Sync the spikes/lfp/nidq
    - make and plot 'bout rasters'

In [1]:
%matplotlib inline
import os
import glob
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal
import traceback
import warnings

from matplotlib import pyplot as plt
from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)


In [2]:
from ceciestunepipe.file import bcistructure as et
from ceciestunepipe.util import sglxutil as sglu
from ceciestunepipe.util import rigutil as ru
from ceciestunepipe.util import wavutil as wu
from ceciestunepipe.util import syncutil as su

from ceciestunepipe.util.sound import boutsearch as bs
from ceciestunepipe.pipeline import searchbout as sb

from ceciestunepipe.util.spikeextractors import preprocess as pre
from ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor import readSGLX as rsgl
from ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor import spikeglxrecordingextractor as sglex

In [3]:
import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.toolkit as st
import spikeinterface.sorters as ss
import spikeinterface.comparison as sc
import spikeinterface.widgets as sw
logger.info('all modules loaded')

2022-04-19 10:29:18,508 root         INFO     all modules loaded


## Session parameters and raw files

#### list all the sessions for this bird

In [4]:
bird = 's_b1555_22'
all_bird_sess = et.list_sessions(bird)
logger.info('all sessions for bird are {}'.format(all_bird_sess))

2022-04-19 10:29:23,452 root         INFO     all sessions for bird are ['2022-03-25', '2022-03-26', '2022-03-27', '2022-03-28', '2022-03-29', '2022-03-30', '2022-03-31', '2022-04-01', '2022-04-02', '2022-04-03', '2022-04-04', '2022-04-05', '2022-04-06', '2022-04-07', '2022-04-08', '2022-04-09', '2022-04-10', '2022-04-11', '2022-04-12', '2022-04-13', '2022-04-14', '2022-04-15', '2022-04-16', '2022-04-17', '2022-04-18', '2022-04-19']


### set up bird and sessions parameters
this will define:
- locations of files (for the bird)
- signals and channels to look for in the metadata of the files and in the rig.json parameter file: Note that this have to exist in all of the sessions that will be processed
- 'sess' is unimportant here, but it comes handy if there is need to debug usin a single session

In [5]:
reload(et)
# for one example session

sess_par = {'bird': bird,
           'sess': '2022-04-18',
           'probes': [], #probes of interest
           'mic_list': ['microphone_0'], #list of mics of interest, by signal name in rig.json
           'adc_list': ['pressure'],
           'nidq_ttl_list': ['wav_ttl'], # list of TTL signals form the nidq digital inputs to extract (besides the 'sync')
           'sort': 2, #label for this sort instance
           }

exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], sort=sess_par['sort'])

ksort_folder = exp_struct['folders']['ksort']
raw_folder = exp_struct['folders']['sglx']

list all the epochs in a session, to check that it is finding what it has to find

In [6]:
sess_epochs = et.list_sgl_epochs(sess_par)
sess_epochs

2022-04-19 10:29:30,792 ceciestunepipe.file.bcistructure INFO     {'folders': {'bird': '/mnt/sphere/speech_bci/raw_data/s_b1555_22', 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1555_22/2022-04-18', 'sglx': '/mnt/sphere/speech_bci/raw_data/s_b1555_22/2022-04-18/sglx', 'kwik': '/scratch/earneodo/s_b1555_22/sglx/kwik/2022-04-18', 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1555_22/2022-04-18/sglx', 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-18/sglx', 'tmp': '/scratch/earneodo/tmp', 'msort': '/scratch/earneodo/s_b1555_22/sglx/msort/2022-04-18', 'ksort': '/scratch/earneodo/s_b1555_22/sglx/ksort/2022-04-18/2', 'sort': '/mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-18/sglx/2'}, 'files': {'par': '/scratch/earneodo/s_b1555_22/sglx/ksort/2022-04-18/2/params.json', 'set': '/mnt/sphere/speech_bci/raw_data/s_b1555_22/2022-04-18/sglx/settings.isf', 'rig': '/mnt/sphere/speech_bci/raw_data/s_b1555_22/2022-04-18/sglx/rig.json', 'kwd': '/scratch/earneodo/s_b155

['0817_g0', '1207_g0', '1530_g0', '1834_g0']

#### define pre-processing steps for each epoch and for the session

In [ ]:
reload(pre)
#one_epoch_dict = pre.preprocess_run(sess_par, exp_struct, sess_epochs[3])

In [7]:
### sequentially process all runs of the sessions
def preprocess_session(sess_par: dict):
    logger.info('pre-process all runs of sess ' + sess_par['sess'])
    # get exp struct
    sess_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], sort=sess_par['sort'])
    # list the epochs
    sess_epochs = et.list_sgl_epochs(sess_par)
    logger.info('found epochs: {}'.format(sess_epochs))
    # preprocess all epochs
    epoch_dict_list = []
    for i_ep, epoch in enumerate(sess_epochs):
        try:
            exp_struct = et.sgl_struct(sess_par, epoch)
            one_epoch_dict = pre.preprocess_run(sess_par, exp_struct, epoch)
            epoch_dict_list.append(one_epoch_dict)
        except Exception as exc:
            warnings.warn('Error in epoch {}'.format(epoch), UserWarning)
            logger.info(traceback.format_exc)
            logger.info(exc)
            logger.info('Session {} epoch {} could not be preprocessed'.format(sess_par['sess'], epoch))
            warnings.warn('Session {} epoch {} could not be preprocessed'.format(sess_par['sess'], epoch))
        
    return epoch_dict_list

all_epoch_list = preprocess_session(sess_par)

2022-04-19 10:29:40,161 root         INFO     pre-process all runs of sess 2022-04-18
2022-04-19 10:29:40,162 ceciestunepipe.file.bcistructure INFO     {'folders': {'bird': '/mnt/sphere/speech_bci/raw_data/s_b1555_22', 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1555_22/2022-04-18', 'sglx': '/mnt/sphere/speech_bci/raw_data/s_b1555_22/2022-04-18/sglx', 'kwik': '/scratch/earneodo/s_b1555_22/sglx/kwik/2022-04-18', 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1555_22/2022-04-18/sglx', 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-18/sglx', 'tmp': '/scratch/earneodo/tmp', 'msort': '/scratch/earneodo/s_b1555_22/sglx/msort/2022-04-18', 'ksort': '/scratch/earneodo/s_b1555_22/sglx/ksort/2022-04-18/2', 'sort': '/mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-18/sglx/2'}, 'files': {'par': '/scratch/earneodo/s_b1555_22/sglx/ksort/2022-04-18/2/params.json', 'set': '/mnt/sphere/speech_bci/raw_data/s_b1555_22/2022-04-18/sglx/settings.isf', 'rig': '/mnt/sphere/spee

2022-04-19 10:31:26,743 ceciestunepipe.util.wavutil INFO     sampling rate 40000
2022-04-19 10:31:26,745 ceciestunepipe.util.wavutil INFO     saving (1, 244632116)-shaped array as wav in /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-18/sglx/1207_g0/wav_sync.wav
2022-04-19 10:31:27,662 ceciestunepipe.util.wavutil INFO     saving (1, 244632116)-shaped array as npy in /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-18/sglx/1207_g0/wav_sync.npy
2022-04-19 10:31:28,558 ceciestunepipe.util.spikeextractors.preprocess INFO     Getting sync events from the wav sync channel
2022-04-19 10:31:29,853 ceciestunepipe.util.spikeextractors.preprocess INFO     saving sync events of the wav channel to /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-18/sglx/1207_g0/wav_sync_evt.npy
2022-04-19 10:31:29,856 ceciestunepipe.util.spikeextractors.preprocess INFO     saving t0 for wav channel to /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-18/sglx/1207_g0/wav_t0.npy
2022-04-19 10

2022-04-19 10:33:40,832 ceciestunepipe.util.spikeextractors.preprocess INFO     Getting adc channel(s) ['pressure']
2022-04-19 10:33:40,837 ceciestunepipe.util.wavutil INFO     sampling rate 40000
2022-04-19 10:33:40,838 ceciestunepipe.util.wavutil INFO     saving (1, 229851904)-shaped array as wav in /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-18/sglx/1834_g0/wav_adc.wav
2022-04-19 10:33:41,882 ceciestunepipe.util.wavutil INFO     saving (1, 229851904)-shaped array as npy in /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-18/sglx/1834_g0/wav_adc.npy
2022-04-19 10:33:50,954 ceciestunepipe.util.spikeextractors.preprocess INFO     Getting sync channel(s) from nidaq streams: ['sync']
2022-04-19 10:33:50,958 ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor.spikeglxrecordingextractor INFO     getting ttl traces, chan [0 1 2 3 4 5 6 7]
2022-04-19 10:33:54,680 ceciestunepipe.util.wavutil INFO     sampling rate 40000
2022-04-19 10:33:54,682 ceciestune

## Process multiple sessions

In [ ]:
sess_list = all_bird_sess
# fist implant, right hemisphere
#sess_list = ['2021-06-24', '2021-06-25', '2021-06-26', '2021-06-27', '2021-06-28', '2021-06-29', '2021-06-30']
sess_list = ['2022-04-16']

In [ ]:
all_sess_dict = {}

for one_sess in sess_list[:]:
    sess_par['sess'] = one_sess
    preprocess_session(sess_par)

## search bouts for those sessions

In [ ]:
hparams = { # default parameters work well for starling
    # spectrogram
    'num_freq':1024, #1024# how many channels to use in a spectrogram #
    'preemphasis':0.97, 
    'frame_shift_ms':5, # step size for fft
    'frame_length_ms':10, #128 # frame length for fft FRAME SAMPLES < NUM_FREQ!!!
    'min_level_db':-55, # minimum threshold db for computing spe 
    'ref_level_db':110, # reference db for computing spec
    'sample_rate':None, # sample rate of your data
    
    # spectrograms
    'mel_filter': False, # should a mel filter be used?
    'num_mels':1024, # how many channels to use in the mel-spectrogram
    'fmin': 500, # low frequency cutoff for mel filter
    'fmax': 12000, # high frequency cutoff for mel filter
    
    # spectrogram inversion
    'max_iters':200,
    'griffin_lim_iters':20,
    'power':1.5,

    # Added for the searching
    'read_wav_fun': bs.read_wav_chan, # function for loading the wav_like_stream (has to returns fs, ndarray)
    'file_order_fun': bs.sess_file_id, # function for extracting the file id within the session
    'min_segment': 30, # Minimum length of supra_threshold to consider a 'syllable' (ms)
    'min_silence': 3000, # Minmum distance between groups of syllables to consider separate bouts (ms)
    'min_bout': 3000, # min bout duration (ms)
    'peak_thresh_rms': 0.55, # threshold (rms) for peak acceptance,
    'thresh_rms': 0.25, # threshold for detection of syllables
    'mean_syl_rms_thresh': 0.3, #threshold for acceptance of mean rms across the syllable (relative to rms of the file)
    'max_bout': 180000, #exclude bouts too long
    'l_p_r_thresh': 100, # threshold for n of len_ms/peaks (typycally about 2-3 syllable spans
    
    'waveform_edges': 1000, #get number of ms before and after the edges of the bout for the waveform sample
    
    'bout_auto_file': 'bout_auto.pickle', # extension for saving the auto found files
    'bout_curated_file': 'bout_checked.pickle', #extension for manually curated files (coming soon)
    }

In [ ]:
all_sessions = sess_list[:]
#all_sessions = ['2021-06-15']

for sess in all_sessions:
    sess_par['sess'] = sess
    sess_bout_pd = sb.get_all_day_bouts(sess_par, hparams, n_jobs=1, ephys_software='sglx')
    sb.save_auto_bouts(sess_bout_pd, sess_par, hparams, software='sglx')
    #sess_bouts_folder = os.path.join(exp_struct['folders']['derived'], 'bouts')
    #bouts_to_wavs(sess_bout_pd, sess_par, hparams, sess_bouts_folder)

In [ ]:
sess_bout_pd.info()

In [ ]:
np.unique(sess_bout_pd['start_ms']).size

# debug

## debug search_bout

In [ ]:
## look for a single file
sess = sess_list[0]

exp_struct = et.get_exp_struct(sess_par['bird'], sess, ephys_software='sglx')
source_folder = exp_struct['folders']['derived']
wav_path_list = et.get_sgl_files_epochs(source_folder, file_filter='*wav_mic.wav')
wav_path_list.sort()
logger.info('Found {} files'.format(len(wav_path_list)))
print(wav_path_list)

In [ ]:
one_file = wav_path_list[0]

In [ ]:
reload(bs)
epoch_bout_pd, epoch_wav = bs.get_bouts_in_long_file(wav_path_list[0], hparams)